In [ ]:
Iris 꽃 분류

* Fully Connected Layer (Dense)를 활용한 분류 모델 (Classification)
* tensorflow-datasets 를 활용한 데이터 전처리

In [2]:
# 필요 모듈 임포트
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# 데이터 셋 로드 및 분할
train_dataset = tfds.load('iris', split='train[:80%]')
valid_dataset = tfds.load('iris', split='train[80%:]')

Shuffling and writing examples to /root/tensorflow_datasets/iris/2.0.0.incompleteHLGSUX/iris-train.tfrecord


Dataset iris downloaded and prepared to /root/tensorflow_datasets/iris/2.0.0. Subsequent calls will reuse this data.


In [6]:
# 데이터 확인 방법
for data in train_dataset.take(10):
    print(data)

{'features': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([5.1, 3.4, 1.5, 0.2], dtype=float32)>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>}
{'features': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([7.7, 3. , 6.1, 2.3], dtype=float32)>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=2>}
{'features': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([5.7, 2.8, 4.5, 1.3], dtype=float32)>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>}
{'features': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([6.8, 3.2, 5.9, 2.3], dtype=float32)>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=2>}
{'features': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([5.2, 3.4, 1.4, 0.2], dtype=float32)>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>}
{'features': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([5.6, 2.9, 3.6, 1.3], dtype=float32)>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>}
{'features': <tf.Tensor: shape=(4,), dtype=float32, numpy=

In [7]:
# 전처리 함수
def preprocess(data):
    x = data['features']
    y = data['label']
    y = tf.one_hot(y, 3)
    return x, y

In [8]:
# 배치사이즈 지정
batch_size=10

In [9]:
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

In [11]:
# 모델 정의
model = tf.keras.models.Sequential([
    # input_shape는 X의 feature 갯수가 4개 이므로 (4, )로 지정합니다.
    Dense(512, activation='relu', input_shape=(4,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    # Classification을 위한 Softmax, 클래스 갯수 = 3개
    Dense(3, activation='softmax'),
])

In [12]:
# 모델 확인
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               2560      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 99        
Total params: 177,219
Trainable params: 177,219
Non-trainable params: 0
________________________________________________

In [13]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [14]:
# 모델 체크포인트
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

In [15]:
# 모델 학습
model.fit(train_data,
          validation_data=(valid_data),
          epochs=20,
          callbacks=[checkpoint],
          )

Epoch 1/20
12/12 [==============================] - 3s 26ms/step - loss: 0.9205 - acc: 0.5667 - val_loss: 0.6994 - val_acc: 0.7000

Epoch 00001: val_loss improved from inf to 0.69938, saving model to my_checkpoint.ckpt
Epoch 2/20
12/12 [==============================] - 0s 10ms/step - loss: 0.5677 - acc: 0.7917 - val_loss: 0.4809 - val_acc: 0.7000

Epoch 00002: val_loss improved from 0.69938 to 0.48087, saving model to my_checkpoint.ckpt
Epoch 3/20
12/12 [==============================] - 0s 11ms/step - loss: 0.3111 - acc: 0.9083 - val_loss: 0.2623 - val_acc: 0.9333

Epoch 00003: val_loss improved from 0.48087 to 0.26233, saving model to my_checkpoint.ckpt
Epoch 4/20
12/12 [==============================] - 0s 11ms/step - loss: 0.2546 - acc: 0.8833 - val_loss: 0.3182 - val_acc: 0.8667

Epoch 00004: val_loss did not improve from 0.26233
Epoch 5/20
12/12 [==============================] - 0s 10ms/step - loss: 0.1448 - acc: 0.9333 - val_loss: 0.1812 - val_acc: 0.9333

Epoch 00005: val_los

In [16]:
# 가중치 로드
model.load_weights(checkpoint_path)